In [152]:
#!/usr/bin/python
# Important Links
# https://github.com/allanbreyes/udacity-data-science/tree/master/p4
# https://github.com/seifip/udacity-data-analyst-nanodegree/tree/master/P5%20-%20Identifying%20Fraud%20from%20Enron%20Emails%20and%20Financial%20Data
# https://www.civisanalytics.com/blog/workflows-in-python-using-pipeline-and-gridsearchcv-for-more-compact-and-comprehensive-code/
# http://sebastianraschka.com/Articles/2014_about_feature_scaling.html
# http://www.datasciencecentral.com/profiles/blogs/feature-scaling-and-normalization
# http://www.programcreek.com/python/example/82501/sklearn.preprocessing.MinMaxScaler
# https://www.analyticsvidhya.com/blog/2016/07/practical-guide-data-preprocessing-python-scikit-learn/
# https://powerbi.microsoft.com/en-us/guided-learning/powerbi-learning-3-11h-r-visual-integration/


In [153]:
import sys
import pickle
import pandas
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
from pandas.tools.plotting import scatter_matrix
plt.style.use('ggplot')
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile, f_classif
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import *

sys.path.append("../tools/")

from feature_format import featureFormat, targetFeatureSplit
from tester import dump_classifier_and_data

In [154]:
# import seaborn as sns

features_list = ['poi']

### Load the dictionary containing the dataset
with open("final_project_dataset.pkl", "r") as data_file:
    data_dict = pickle.load(data_file)
    
my_dataset = data_dict

In [155]:
# # enron_df = pandas.DataFrame.from_dict(my_dataset)
# for i in data_dict.values():
#     plt.scatter( i['salary'] , i['total_payments']  )

# plt.xlabel("salary")
# plt.ylabel("total_payments")
# plt.show()

In [156]:
# # plt.show()
# for k, v in data_dict.items():
#     if v['salary'] != 'NaN' and v['salary'] > 1111258: print k

In [157]:
# for i in data_dict.values():
#     plt.scatter( i['salary'] , i['bonus']  )

# plt.xlabel("salary")
# plt.ylabel("bonus")
# plt.show()

# # there is an extreme outlier



# # turns out to be the TOTAL row from the salaries & as.read_csv("enron.csv")
# # enron_df.head()bonuses list, let's remove it


In [158]:
del data_dict["TOTAL"]

In [159]:
my_dataset = data_dict

In [160]:
## Communication:
for item in my_dataset:
    person = my_dataset[item]
    if (all([person['from_poi_to_this_person'] != 'NaN',
             person['from_this_person_to_poi'] != 'NaN',
             person['to_messages'] != 'NaN',
             person['from_messages'] != 'NaN'
            ])):
        fraction_from_poi = float(person["from_poi_to_this_person"]) / float(person["to_messages"])
        person["fraction_from_poi"] = fraction_from_poi
        fraction_to_poi = float(person["from_this_person_to_poi"]) / float(person["from_messages"])
        person["fraction_to_poi"] = fraction_to_poi
    else:
        person["fraction_from_poi"] = person["fraction_to_poi"] = 0

# ## Financial:
for item in my_dataset:
    person = my_dataset[item]
    if (all([person['salary'] != 'NaN',
             person['total_stock_value'] != 'NaN',
             person['exercised_stock_options'] != 'NaN',
             person['bonus'] != 'NaN'
            ])):
        person['wealth'] = sum([person[field] for field in ['salary',
                                                            'total_stock_value',
                                                            'exercised_stock_options',
                                                            'bonus']])
    else:
        person['wealth'] = 'NaN'

my_features = features_list + ['fraction_from_poi',
                               'fraction_to_poi',
                               'shared_receipt_with_poi',
                               'expenses',
                               'loan_advances',
                               'long_term_incentive',
                               'other',
                               'restricted_stock',
                               'restricted_stock_deferred',
                               'deferral_payments',
                               'deferred_income',
                               'salary',
                               'total_stock_value',
                               'exercised_stock_options',
                               'total_payments',
                               'bonus',
                               'wealth']

In [161]:
### Extract features and labels from dataset for local testing
data = featureFormat(my_dataset, my_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

In [170]:
features

array([[ 0.07451757,  0.02961276,  0.25484514, ...,  0.04330293,
         0.521875  ,  0.08561095],
       [ 0.        ,  0.        ,  0.        , ...,  0.00176194,
         0.        ,  0.        ],
       [ 0.31703443,  0.        ,  0.08422387, ...,  0.00884703,
         0.        ,  0.        ],
       ..., 
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.00347915,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.00053203,
         0.        ,  0.        ]])

In [162]:
# # Scale features
scaler = MinMaxScaler()
features = scaler.fit_transform(features)

In [163]:
print "Intuitive features:", my_features

Intuitive features: ['poi', 'fraction_from_poi', 'fraction_to_poi', 'shared_receipt_with_poi', 'expenses', 'loan_advances', 'long_term_incentive', 'other', 'restricted_stock', 'restricted_stock_deferred', 'deferral_payments', 'deferred_income', 'salary', 'total_stock_value', 'exercised_stock_options', 'total_payments', 'bonus', 'wealth']


# Gaussian

In [164]:
select = SelectKBest()
# pca = PCA(n_components = 7)
clf = GaussianNB()

steps = [('feature_selection', select),
        ('GaussianNB', clf)]

pipeline = Pipeline(steps)

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

# d_features = pca.fit_transform(features_train)
# ds_features = pca.transform(features_test)

### and print the report
parameters = dict(feature_selection__k = [3,4,5])

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 10)

### fit your pipeline on features_train and labels_train
cv.fit(features_train, labels_train)

### call pipeline.predict() on your features_test data to make a set of test predictions
pred = cv.predict(features_test)

### test your predictions using sklearn.classification_report()
report = classification_report(labels_test, pred)
accuracy = accuracy_score(labels_test, pred)

results = pandas.DataFrame.from_dict(cv.cv_results_)

results_list = zip(cv.best_estimator_.named_steps['feature_selection'].get_support(), my_features[1:],
                   cv.best_estimator_.named_steps['feature_selection'].scores_)

results_list = sorted(results_list, key = lambda x: x[2], reverse=True)

print(cv.best_estimator_)

best_scores = cv.best_estimator_.named_steps['feature_selection'].scores_
best_scores = sorted([round(x, 3) for x in best_scores], reverse = True)

# print best_scores
# results_list
# print results
for i in results_list: print i
print '\n',cv.best_estimator_.steps[1][1]
print'\n',report
print '\n', accuracy

Pipeline(steps=[('feature_selection', SelectKBest(k=3, score_func=<function f_classif at 0x7ff60d37bb90>)), ('GaussianNB', GaussianNB(priors=None))])
(True, 'bonus', 29.081491199795181)
(True, 'salary', 15.119692725298126)
(True, 'fraction_to_poi', 12.89761482372556)
(False, 'total_stock_value', 10.374045713087989)
(False, 'shared_receipt_with_poi', 9.8582776246375587)
(False, 'exercised_stock_options', 9.6305853837525035)
(False, 'wealth', 8.9660211199691791)
(False, 'total_payments', 8.5140956287314875)
(False, 'long_term_incentive', 8.1194130541475662)
(False, 'deferred_income', 7.9221360027191352)
(False, 'restricted_stock', 7.5855209858935693)
(False, 'loan_advances', 6.7057870201376479)
(False, 'expenses', 3.8744660455757707)
(False, 'other', 2.9463988912624939)
(False, 'restricted_stock_deferred', 0.71324401933397774)
(False, 'fraction_from_poi', 0.37577092640120102)
(False, 'deferral_payments', 0.019933736026878885)

GaussianNB(priors=None)

             precision    recall  f1

# Random Forest

In [165]:
data = featureFormat(my_dataset, my_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

select = SelectKBest()
# pca = PCA(n_components = 7)
clf = RandomForestClassifier()

steps = [('feature_selection', select),
        ('random_forest', clf)]

pipeline = Pipeline(steps)

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

# d_features = pca.fit_transform(features_train)
# ds_features = pca.transform(features_test)

### and print the report
parameters = dict(feature_selection__k = [3,4,5],
                  random_forest__n_estimators=[50, 100, 200],
                  random_forest__min_samples_split=[2, 3, 4, 5, 10])

cv = GridSearchCV(pipeline, param_grid = parameters)

### fit your pipeline on features_train and labels_train
cv.fit(features_train, labels_train)

### call pipeline.predict() on your features_test data to make a set of test predictions
pred = cv.predict(features_test)

### test your predictions using sklearn.classification_report()
report = classification_report(labels_test, pred)
accuracy = accuracy_score(labels_test, pred)

results = pandas.DataFrame.from_dict(cv.cv_results_)

results_list = zip(cv.best_estimator_.named_steps['feature_selection'].get_support(), my_features[1:],
                   cv.best_estimator_.named_steps['feature_selection'].scores_)

results_list = sorted(results_list, key = lambda x: x[2], reverse=True)

print(cv.best_estimator_)

best_scores = cv.best_estimator_.named_steps['feature_selection'].scores_
best_scores = sorted([round(x, 3) for x in best_scores], reverse = True)

# print best_scores
for i in results_list: print i
print '\n',cv.best_estimator_.steps[1][1]
print'\n',report
print'\n',accuracy
# print results

Pipeline(steps=[('feature_selection', SelectKBest(k=5, score_func=<function f_classif at 0x7ff60d37bb90>)), ('random_forest', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07,...mators=100, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))])
(True, 'bonus', 29.081491199795181)
(True, 'salary', 15.119692725298126)
(True, 'fraction_to_poi', 12.89761482372556)
(True, 'total_stock_value', 10.374045713087989)
(True, 'shared_receipt_with_poi', 9.8582776246375587)
(False, 'exercised_stock_options', 9.6305853837525035)
(False, 'wealth', 8.9660211199691791)
(False, 'total_payments', 8.5140956287314875)
(False, 'long_term_incentive', 8.1194130541475662)
(False, 'deferred_income', 7.9221360027191352)
(False, 'restricted_stock', 7.5855209858935693)
(False, 'loan_advances', 6.7057870201376479)
(False, 'expenses', 3.87446604557577

# SVC

In [166]:
data = featureFormat(my_dataset, my_features, sort_keys = True)
labels, features = targetFeatureSplit(data)

scaler = MinMaxScaler()
features = scaler.fit_transform(features)

select = SelectKBest()
# pca = PCA(n_components = 7)
clf = SVC()

steps = [('feature_selection', select),
        ('svc', clf)]

pipeline = Pipeline(steps)

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.33, random_state=42)

d_features = pca.fit_transform(features_train)
ds_features = pca.transform(features_test)

### print the report
parameters = dict(feature_selection__k = [3,4,5],
                  svc__kernel = ["rbf", "linear"],
                  svc__C = [1, 10, 50, 100, 200])

cv = GridSearchCV(pipeline, param_grid = parameters, cv = 10)

### fit pipeline on features_train and labels_train
cv.fit(features_train, labels_train)

### call pipeline.predict() on your features_test data to make a set of test predictions
pred = cv.predict(features_test)

### test predictions using sklearn.classification_report()
report = classification_report( labels_test, pred)
accuracy = accuracy_score(labels_test, pred)

results = pandas.DataFrame.from_dict(cv.cv_results_)

results_list = zip(cv.best_estimator_.named_steps['feature_selection'].get_support(), my_features[1:],
                   cv.best_estimator_.named_steps['feature_selection'].scores_)

results_list = sorted(results_list, key = lambda x: x[2], reverse=True)

# print(cv.best_estimator_)

best_scores = cv.best_estimator_.named_steps['feature_selection'].scores_
best_scores = sorted([round(x, 3) for x in best_scores], reverse = True)

# print best_scores
for i in results_list: print i
print '\n', cv.best_estimator_.steps[1][1]
print '\n', accuracy
print '\n', report

# print results


(True, 'bonus', 29.081491199795181)
(True, 'salary', 15.119692725298126)
(True, 'fraction_to_poi', 12.89761482372556)
(False, 'total_stock_value', 10.374045713087989)
(False, 'shared_receipt_with_poi', 9.8582776246375587)
(False, 'exercised_stock_options', 9.6305853837525035)
(False, 'wealth', 8.9660211199691791)
(False, 'total_payments', 8.5140956287314875)
(False, 'long_term_incentive', 8.1194130541475662)
(False, 'deferred_income', 7.9221360027191352)
(False, 'restricted_stock', 7.5855209858935693)
(False, 'loan_advances', 6.7057870201376479)
(False, 'expenses', 3.8744660455757707)
(False, 'other', 2.9463988912624939)
(False, 'restricted_stock_deferred', 0.71324401933397774)
(False, 'fraction_from_poi', 0.37577092640120102)
(False, 'deferral_payments', 0.019933736026878885)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, ver

## giving preference to recall over precision hence SVC

In [167]:
my_features = features_list + ['fraction_to_poi','salary','bonus']

In [168]:
clf = SVC(C = 10, kernel = 'rbf')

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size=0.525, random_state=42)

clf.fit(features_train, labels_train)
pred = clf.predict(features_test)

report = classification_report(labels_test, pred)
accuracy = accuracy_score(labels_test, pred)


from sklearn.model_selection import cross_val_score
scores = cross_val_score(clf, features, labels, cv = 5)

print '\n', scores
print '\n', scores.mean()

print '\n', "Accuracy:", accuracy
print '\n', "Report:", report


[ 0.86666667  0.82758621  0.86206897  0.89285714  0.89285714]

0.868407224959

Accuracy: 0.868421052632

Report:              precision    recall  f1-score   support

        0.0       0.88      0.98      0.93        66
        1.0       0.50      0.10      0.17        10

avg / total       0.83      0.87      0.83        76



In [169]:
dump_classifier_and_data(clf, my_dataset, my_features)